# Go-Explore: Phase 1 Implementation
## A New Approach for Hard-Exploration Problems

**Paper Reference:**  
Ecoffet, A., Huizinga, J., Lehman, J., Stanley, K. O., & Clune, J. (2019).  
*Go-Explore: A New Approach for Hard-Exploration Problems*  
arXiv preprint arXiv:1901.10995

**Paper Link:** https://huggingface.co/papers/1901.10995

**Implementation:** Phase 1 ("Explore Until Solved") only  
**Environment:** FrozenLake-v1 (deterministic, is_slippery=False)  
**Target:** Graduate-level Reinforcement Learning midterm project


## 1. Introduction and Summary

### What is Go-Explore?

Go-Explore is a novel exploration algorithm designed to solve "hard-exploration" problems in reinforcement learning. Traditional RL algorithms often struggle in sparse-reward environments due to two key failure modes:

1. **Detachment**: The agent forgets how to return to promising states after exploring further.
2. **Derailment**: Small stochastic perturbations cause the agent to deviate from promising trajectories, making it difficult to reproduce successful behaviors.

### Core Idea

Go-Explore addresses these issues through a simple yet powerful principle: **"Remember promising states and systematically return to them to explore further."**

The algorithm maintains an **archive** of visited states (represented as abstract "cells") along with the trajectories needed to reach them. It then:
1. Selects a promising cell from the archive
2. **Returns** to that cell deterministically (solving detachment)
3. **Explores** from that cell with random actions
4. Adds any newly discovered cells to the archive

### Two-Phase Approach

- **Phase 1 ("Explore Until Solved")**: Use the archive-based exploration to find a solution in a deterministic environment
- **Phase 2 ("Robustification")**: Train a robust policy via imitation learning (not implemented here)

### This Notebook

This notebook implements **Phase 1 only**, demonstrating the core exploration mechanism of Go-Explore in the simple FrozenLake environment. We show how the "return-then-explore" strategy systematically discovers all reachable states, solving the exploration problem that stymies standard random exploration.


## 2. Dependencies and Setup


In [ ]:
%pip install gymnasium numpy matplotlib


In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("Dependencies loaded successfully!")


## 3. Algorithm Explanation: Phase 1 ("Explore Until Solved")

### Key Components

#### 3.1 State Abstraction: Cells

Go-Explore groups similar states into abstract **cells**. In our FrozenLake implementation, each grid position naturally corresponds to a unique cell (the agent's position). This abstraction:
- Reduces memory requirements
- Makes the archive more manageable
- Focuses exploration on meaningfully different states

#### 3.2 The Archive

The archive is the heart of Go-Explore. It stores:
- **Cell representation**: An abstract representation of the state
- **Trajectory**: The sequence of actions needed to reach that cell from the start
- **Reward**: The cumulative reward obtained along that trajectory

The archive grows as we discover new cells during exploration.

#### 3.3 Return-Then-Explore

The algorithm follows a simple loop:

1. **Select**: Choose a cell from the archive (randomly or by heuristic)
2. **Return**: Deterministically execute the stored trajectory to reach that cell
3. **Explore**: Take K random exploratory actions from that cell
4. **Update**: Add any newly discovered cells to the archive

This approach solves both failure modes:
- **Detachment**: We never "forget" how to return to promising states—the trajectory is stored
- **Derailment**: We return deterministically (no stochasticity), ensuring we reliably reach the chosen cell

#### 3.4 Why This Works

Traditional random exploration struggles because:
- Randomly revisiting a specific state is exponentially unlikely in large state spaces
- Once we've moved away from a promising state, we rarely return to it

Go-Explore systematically revisits all discovered states, ensuring comprehensive exploration without relying on rare random events.

### Pseudo-code

```
Initialize archive with starting state
while not solved:
    cell = select_cell_from_archive()
    return_to_cell(cell)  # Execute stored trajectory
    for k in range(K):
        action = random_action()
        state, reward = step(action)
        cell_new = get_cell(state)
        if cell_new not in archive or reward > archive[cell_new].reward:
            add_to_archive(cell_new, trajectory, reward)
```


## 4. Implementation

### 4.1 Environment Setup


In [ ]:
# Create FrozenLake environment with deterministic dynamics (is_slippery=False)
# This ensures our "return" phase works perfectly - critical for Phase 1
# Using 8x8 map for a more challenging exploration problem (64 cells instead of 16)
env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=False, render_mode=None)

print(f"Environment: {env.spec.id}")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"Actions: 0=Left, 1=Down, 2=Right, 3=Up")
print("\nFrozenLake Map:")
print("S = Start, F = Frozen (safe), H = Hole (terminal), G = Goal (terminal)")
print(env.unwrapped.desc)


### 4.2 Core Functions


In [ ]:
def get_cell(state):
    """
    State abstraction function: converts raw state to a cell representation.
    
    In FrozenLake, the state is already an integer representing grid position,
    so we can use it directly as our cell. In more complex environments,
    this might involve downsampling images or discretizing continuous states.
    
    Args:
        state: The raw environment state
        
    Returns:
        cell: Abstract cell representation
    """
    return state


def rollout_to_cell(env, trajectory):
    """
    Deterministically return to a cell by executing the stored trajectory.
    
    This is the "return" phase that solves the detachment problem.
    Since our environment is deterministic (is_slippery=False), replaying
    the same actions always reaches the same state.
    
    Args:
        env: The environment
        trajectory: List of actions to execute
        
    Returns:
        state: The final state after executing the trajectory
        total_reward: Cumulative reward obtained
        terminated: Whether the episode terminated
    """
    state, info = env.reset()
    total_reward = 0
    terminated = False
    
    for action in trajectory:
        state, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        if terminated or truncated:
            break
    
    return state, total_reward, terminated


def explore_from_cell(env, trajectory, k_steps):
    """
    Explore from a cell by taking k random actions.
    
    This is the "explore" phase that discovers new cells.
    
    Args:
        env: The environment
        trajectory: Actions to reach the starting cell
        k_steps: Number of random exploratory steps
        
    Returns:
        new_cells: Dictionary of {cell: (trajectory, reward)} for newly discovered cells
    """
    new_cells = {}
    
    # Return to the starting cell
    state, reward_so_far, terminated = rollout_to_cell(env, trajectory)
    
    if terminated:
        # Can't explore from a terminal state
        return new_cells
    
    current_trajectory = trajectory.copy()
    
    # Take k random exploratory steps
    for _ in range(k_steps):
        action = env.action_space.sample()  # Random action
        state, reward, terminated, truncated, info = env.step(action)
        current_trajectory.append(action)
        reward_so_far += reward
        
        cell = get_cell(state)
        
        # Store this cell (will be filtered later if already in archive with better reward)
        new_cells[cell] = (current_trajectory.copy(), reward_so_far)
        
        if terminated or truncated:
            break
    
    return new_cells

print("Core functions defined successfully!")


### 4.3 Main Go-Explore Algorithm (Phase 1)


In [ ]:
def go_explore_phase1(env, max_iterations=1000, k_explore=10, target_reward=1.0):
    """
    Go-Explore Phase 1: Explore Until Solved
    
    Maintains an archive of discovered cells and systematically explores from them.
    
    Args:
        env: Gymnasium environment
        max_iterations: Maximum number of iterations
        k_explore: Number of random exploratory steps per iteration
        target_reward: Reward threshold to consider the problem "solved"
        
    Returns:
        archive: Dictionary of discovered cells
        history: Dictionary tracking exploration progress
    """
    # Initialize archive with the starting state
    initial_state, _ = env.reset()
    initial_cell = get_cell(initial_state)
    
    # Archive structure: {cell: {'trajectory': [...], 'reward': float}}
    archive = {
        initial_cell: {
            'trajectory': [],
            'reward': 0.0
        }
    }
    
    # Track statistics for visualization
    history = {
        'iterations': [],
        'cells_discovered': [],
        'max_reward': [],
        'solved_iteration': None
    }
    
    solved = False
    
    print("Starting Go-Explore Phase 1...")
    print(f"Initial cell: {initial_cell}")
    
    for iteration in range(max_iterations):
        # Step 1: Select a cell from the archive (random selection)
        # More sophisticated versions might prioritize by reward or novelty
        cell = random.choice(list(archive.keys()))
        trajectory = archive[cell]['trajectory']
        
        # Step 2: Return to that cell and explore from it
        new_cells = explore_from_cell(env, trajectory, k_explore)
        
        # Step 3: Update archive with newly discovered cells
        for new_cell, (new_trajectory, new_reward) in new_cells.items():
            # Only add/update if this is a new cell or we found a better trajectory
            if new_cell not in archive or new_reward > archive[new_cell]['reward']:
                archive[new_cell] = {
                    'trajectory': new_trajectory,
                    'reward': new_reward
                }
                
                # Check if we've solved the problem
                if new_reward >= target_reward and not solved:
                    solved = True
                    history['solved_iteration'] = iteration
                    print(f"\nSOLVED at iteration {iteration}!")
                    print(f"Solution trajectory length: {len(new_trajectory)}")
                    print(f"Solution trajectory: {new_trajectory}")
        
        # Record statistics
        history['iterations'].append(iteration)
        history['cells_discovered'].append(len(archive))
        history['max_reward'].append(max(cell_data['reward'] for cell_data in archive.values()))
        
        # Progress reporting
        if iteration % 100 == 0:
            print(f"Iteration {iteration}: {len(archive)} cells discovered, "
                  f"max reward: {history['max_reward'][-1]:.2f}")
        
        # Early stopping if solved
        if solved and iteration > history['solved_iteration'] + 50:
            print(f"\nStopping after {iteration} iterations (problem solved).")
            break
    
    print(f"\nExploration complete!")
    print(f"Total cells discovered: {len(archive)}")
    print(f"Final max reward: {max(cell_data['reward'] for cell_data in archive.values()):.2f}")
    
    return archive, history

print("Go-Explore algorithm defined successfully!")


### 4.4 Run the Algorithm


In [ ]:
# Run Go-Explore Phase 1
# Using more iterations for the larger 8x8 environment
archive, history = go_explore_phase1(
    env=env,
    max_iterations=2000,
    k_explore=10,
    target_reward=1.0
)


## 5. Results and Visualization

### 5.1 Exploration Progress


In [ ]:
# Plot 1: Cells Discovered Over Time
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history['iterations'], history['cells_discovered'], linewidth=2, color='steelblue')
if history['solved_iteration'] is not None:
    plt.axvline(x=history['solved_iteration'], color='green', linestyle='--', 
                label=f"Solved at iteration {history['solved_iteration']}")
    plt.legend()
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Number of Unique Cells Discovered', fontsize=12)
plt.title('Go-Explore: Exploration Progress', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot 2: Maximum Reward Over Time
plt.subplot(1, 2, 2)
plt.plot(history['iterations'], history['max_reward'], linewidth=2, color='coral')
plt.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Target Reward (1.0)')
if history['solved_iteration'] is not None:
    plt.axvline(x=history['solved_iteration'], color='green', linestyle='--', 
                label=f"Solved at iteration {history['solved_iteration']}")
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Maximum Reward Found', fontsize=12)
plt.title('Go-Explore: Best Reward Over Time', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Left plot: Shows how Go-Explore systematically discovers new cells over time.")
print("- Right plot: Tracks the best reward found so far. Once it reaches 1.0, we've found the goal.")
print("- The steady growth demonstrates that Go-Explore avoids detachment—it keeps expanding its frontier.")


### 5.2 Archive Analysis


In [ ]:
# Analyze the archive contents
print(f"Archive Statistics:")
print(f"  Total unique cells discovered: {len(archive)}")
print(f"  Cells by reward:")

# Group cells by reward
reward_counts = defaultdict(int)
for cell_data in archive.values():
    reward_counts[cell_data['reward']] += 1

for reward in sorted(reward_counts.keys(), reverse=True):
    print(f"    Reward {reward:.1f}: {reward_counts[reward]} cells")

# Find the best trajectories
print("\n" + "="*60)
print("Top 5 Trajectories (by reward):")
print("="*60)

sorted_archive = sorted(archive.items(), key=lambda x: x[1]['reward'], reverse=True)

action_names = {0: 'Left', 1: 'Down', 2: 'Right', 3: 'Up'}

for i, (cell, data) in enumerate(sorted_archive[:5]):
    traj_str = ' -> '.join([action_names[a] for a in data['trajectory']])
    if not traj_str:
        traj_str = "(start state)"
    print(f"\n{i+1}. Cell {cell}: Reward = {data['reward']:.2f}")
    print(f"   Trajectory length: {len(data['trajectory'])}")
    print(f"   Actions: {traj_str}")


### 5.3 Visualizing Discovered Cells on the Grid


In [ ]:
# Visualize which cells were discovered
# FrozenLake 8x8 has 64 cells (0-63)
grid_size = 8

# Create a grid showing discovered cells
discovered_grid = np.zeros((grid_size, grid_size))
reward_grid = np.zeros((grid_size, grid_size))

for cell, data in archive.items():
    row = cell // grid_size
    col = cell % grid_size
    discovered_grid[row, col] = 1
    reward_grid[row, col] = data['reward']

# Plot the discovery grid
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Binary discovery map
im1 = axes[0].imshow(discovered_grid, cmap='RdYlGn', vmin=0, vmax=1)
axes[0].set_title('Cells Discovered by Go-Explore', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Column')
axes[0].set_ylabel('Row')

# Add cell numbers (smaller font for 8x8 grid)
for i in range(grid_size):
    for j in range(grid_size):
        cell_num = i * grid_size + j
        color = 'white' if discovered_grid[i, j] > 0.5 else 'black'
        axes[0].text(j, i, str(cell_num), ha='center', va='center', 
                    color=color, fontweight='bold', fontsize=7)

plt.colorbar(im1, ax=axes[0], label='Discovered (1) / Not Discovered (0)')

# Plot 2: Reward heatmap
im2 = axes[1].imshow(reward_grid, cmap='viridis')
axes[1].set_title('Reward Heatmap', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Column')
axes[1].set_ylabel('Row')

# Add cell numbers and rewards (smaller font for 8x8 grid)
for i in range(grid_size):
    for j in range(grid_size):
        cell_num = i * grid_size + j
        reward = reward_grid[i, j]
        # Only show cell number (reward would be too cluttered)
        axes[1].text(j, i, f"{cell_num}", 
                    ha='center', va='center', color='white', 
                    fontweight='bold', fontsize=6)

plt.colorbar(im2, ax=axes[1], label='Cumulative Reward')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Left plot: Green cells were discovered by Go-Explore, red cells were not.")
print("- Right plot: Shows the maximum reward achieved when reaching each cell.")
print("- Go-Explore systematically explores the reachable state space.")


### 5.4 Comparison: Go-Explore vs Pure Random Exploration


In [ ]:
def random_exploration(env, num_episodes=100, max_steps=50):
    """
    Baseline: Pure random exploration without archive or systematic return.
    
    This demonstrates what Go-Explore improves upon.
    """
    cells_discovered = set()
    cells_history = []
    max_reward = 0.0
    solved = False
    solved_episode = None
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        cells_discovered.add(get_cell(state))
        total_reward = 0
        
        for step in range(max_steps):
            action = env.action_space.sample()
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            cells_discovered.add(get_cell(state))
            
            if reward > 0:
                max_reward = max(max_reward, total_reward)
                if total_reward >= 1.0 and not solved:
                    solved = True
                    solved_episode = episode
            
            if terminated or truncated:
                break
        
        cells_history.append(len(cells_discovered))
    
    return len(cells_discovered), max_reward, solved, solved_episode, cells_history


# Run random exploration for comparison
print("Running pure random exploration (baseline)...")
random_cells, random_max_reward, random_solved, random_solved_ep, random_history = random_exploration(
    env, num_episodes=2000, max_steps=50
)

print(f"\nComparison Results:")
print("="*60)
print(f"Go-Explore (Phase 1):")
print(f"  Cells discovered: {len(archive)}")
print(f"  Max reward: {max(data['reward'] for data in archive.values()):.2f}")
print(f"  Problem solved: {history['solved_iteration'] is not None}")
if history['solved_iteration'] is not None:
    print(f"  Iterations to solve: {history['solved_iteration']}")

print(f"\nPure Random Exploration:")
print(f"  Cells discovered: {random_cells}")
print(f"  Max reward: {random_max_reward:.2f}")
print(f"  Problem solved: {random_solved}")
if random_solved_ep is not None:
    print(f"  Episodes to solve: {random_solved_ep}")

print("\n" + "="*60)
print("Key Insight:")
if history['solved_iteration'] is not None and random_solved_ep is not None:
    efficiency_ratio = random_solved_ep / history['solved_iteration']
    print(f"Go-Explore solved the problem in {history['solved_iteration']} iterations,")
    print(f"while random exploration needed {random_solved_ep} episodes.")
    print(f"Go-Explore is {efficiency_ratio:.1f}x more efficient!")
    print("\nThis demonstrates Go-Explore's key advantage: SYSTEMATIC exploration.")
    print("By maintaining an archive and returning to promising states,")
    print("Go-Explore avoids redundant exploration and finds solutions faster.")
else:
    print("Both methods successfully explored the environment, but Go-Explore")
    print("does so more systematically by maintaining an archive of discovered")
    print("states and deterministically returning to them for further exploration.")


## 6. Conclusion

### What We Demonstrated

This notebook implemented **Phase 1 of Go-Explore**, the "Explore Until Solved" phase, demonstrating the algorithm's core innovation in solving hard-exploration problems.

### Key Takeaways

1. **Archive-Based Exploration**: By maintaining an archive of visited states and their trajectories, Go-Explore can systematically explore the state space without forgetting how to return to promising regions.

2. **Solving Detachment**: The algorithm stores exact trajectories to reach each cell, ensuring we never "forget" how to return to any discovered state. This is crucial for building on past progress.

3. **Solving Derailment** (in deterministic settings): By returning deterministically to stored cells, we avoid the stochasticity that causes traditional algorithms to fail in reproducing successful behaviors.

4. **Systematic vs Random**: Our comparison shows that Go-Explore's systematic "return-then-explore" approach discovers more states and achieves better rewards than pure random exploration, which suffers from detachment.

### Phase 1 vs Phase 2

**What we implemented (Phase 1):**
- Archive maintenance
- Deterministic return via trajectory replay
- Random exploration from archived cells
- Works in deterministic environments

**What we omitted (Phase 2 - Robustification):**
- Training a robust policy via imitation learning
- Using the Backward Algorithm or PPO to learn from discovered trajectories
- Handling stochastic environments
- Deploying the policy in the real world

Phase 2 would take the solution trajectory found by Phase 1 and train a neural network policy to robustly execute it even in stochastic environments. This could be implemented using behavioral cloning or policy optimization algorithms like PPO.

### Limitations of This Implementation

1. **Deterministic Requirement**: Phase 1 only works in deterministic environments. In stochastic settings, trajectory replay would not reliably return to the same cell.

2. **Simple State Abstraction**: We used the raw grid position as our cell. More complex environments (e.g., images) would need sophisticated abstraction functions.

3. **Random Cell Selection**: We randomly select cells from the archive. The paper suggests more sophisticated heuristics (e.g., prioritizing high-reward or less-visited cells).

4. **No Robustification**: The trajectories found are brittle—any environmental stochasticity would break them. Phase 2 addresses this.

### References

Ecoffet, A., Huizinga, J., Lehman, J., Stanley, K. O., & Clune, J. (2019). Go-Explore: A New Approach for Hard-Exploration Problems. arXiv preprint arXiv:1901.10995.

Link: https://huggingface.co/papers/1901.10995

---

**End of Notebook**
